# Lab: Mounting Azure Blob Storage
---
This Lab will show you how to *mount* one of the most popular big data stores on Azure: Azure Blob Storage
  
*Mounting* Storage in Databricks refers to the process of establishing a connection between your Azure Databricks workspace and other Azure Storage Services. Mounting them allows them to be accessible via tools like `dbutils`. Additionally, and more importantly, it makes interacting with data stores on Azure simpler.

First, let's take a look at Mounting Azure Blob Storage.  
  
However, we need a Azure Blob Storage instance before we continue (if you haven't provisioned Azure Blob Storage, do so now):
  
Within the Azure Portal you need to grab two pieces of information:
1. The Endpoint for the blob with the format of 'wasbs://[your-container-name]@[your-storage-account-name].blob.core.windows.net'
2. The value associated with one of the Access Keys

In [0]:
# Name of the Storage Account
blobAccountName = "<ENTER YOUR STORAGE ACCOUNT NAME>"

# Name of the Container in the Blob Storage Account
containerName = "<ENTER IN THE NAME OF THE CONTAINER>"

# the second piece of information described above goes into blobKey
blobKey = "<ENTER IN THE STORAGE ACCOUNT ACCESS KEY>"

# the first piece of information described above goes into blobEndpoint
blobEndpoint = f"wasbs://{containerName}@{blobAccountName}.blob.core.windows.net/"

Below is a command to mount the Azure Blob Storage container within your Databricks workspace.  
**Source**: The endpoint for your blob  
**mount_point**: The filepath that you will use to access and interact with the mounted storage within Azure Databricks  
**extra_configs**: Here, the only extra configuration you need is the Access Key value

## Explanation of extra_configs and How it should be constructed
In extra_configs, Databricks needs the key-value pair of "fs.azure.sas.CONTAINER_NAME_HERE.ACCOUNT_NAME_HERE.blob.core.windows.net" : blobKey  
  
The access key (i.e. the variable blobKey) should have already been provided in the above cell  
  
The config "fs.azure.sas.CONTAINER_NAME_HERE.ACCOUNT_NAME_HERE.blob.core.windows.net" is telling Databricks what kind of credential you are using for authenticating the connection to the storage account  
  
To fill this out you need to supply the Storage Account Name and Container Nmae. For example, if you deployed a Storage Account with the name of **workshopstorageaccount** and a container with the name **workshopcontainer** then your configuration would look like fs.azure.sas.workshopcontainer.workshopstorageaccount.blob.core.windows.net 
  
The full `extra_configs` would look like the following: `extra_configs = {"fs.azure.sas.workshopcontainer.workshopstorageaccount.blob.core.windows.net":blobKey}`

In [0]:
configKey = f"fs.azure.sas.{containerName}.{blobAccountName}.blob.core.windows.net"

dbutils.fs.mount(
  source = blobEndpoint,
  mount_point = "/mnt/blobmount",
  extra_configs = {
    configKey:blobKey
  }
)

Now lets take a look and make sure it is all connected.

In [0]:
dbutils.fs.ls("/mnt/blobmount")

Another useful feature is the `refreshMounts` command that is accessible via the `dbutils` commands. This refreshes the connection between your Databricks workspace and the storage locations on Azure. It is a good way of ensuring that you are still connected to the mounts.

In [0]:
dbutils.fs.refreshMounts()

Before we finish with this Lab, let's take a look at how to *unmount* storage locations from your Databricks workspace.

In [0]:
dbutils.fs.unmount("/mnt/blobmount")